In [23]:
from itertools import islice
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.food_db

In [25]:
a = [{
     '$lookup':
       {
         'from': "prisma",
         'localField': "name",
         'foreignField': "name",
         'as': "prisma"
       }
  },
  {
  '$match': {'prisma': { '$not': {'$size': 0} }},
  },
{
     '$lookup':
       {
         'from': "metro",
         'localField': "name",
         'foreignField': "name",
         'as': "metro"
       }
  },
  {
  '$match': {'metro': { '$not': {'$size': 0} }},
  },
{
     '$lookup':
       {
         'from': "carusel",
         'localField': "name",
         'foreignField': "name",
         'as': "carusel"
       }
  },
  {
  '$match': {'carusel': { '$not': {'$size': 0} }},
  },
  {
  '$project': {'_id': 0, 'data-amount':0, 'url': 0},
  }        

]
it = db.lenta.aggregate(a)
data = list(it)

In [26]:
import csv
with open('merged.csv', 'w') as f:
    w = csv.writer(f, delimiter="\t")
    w.writerow('name lenta prisma metro carusel lenta prisma metro carusel'.split())
    for i in data:
        w.writerow([i['name'], i['price'], 
                    i['prisma'][0]['price'],
                    i['metro'][0]['price'],
                    i['carusel'][0]['price'],
                    i.get('data-old-price', None), 
                    i['prisma'][0].get('data-old-price', None), 
                    i['metro'][0].get('data-old-price', None), 
                    i['carusel'][0].get('data-old-price', None), 
                    i['weight'], 
                    i['prisma'][0]['weight'], 
                    i['metro'][0]['weight'], 
                    i['carusel'][0]
                    ['weight'], 
                    ])



In [30]:
import csv
with open('prisma.csv', 'w') as f:
    w = csv.DictWriter(f, delimiter="\t", fieldnames=['name', 'item_weight','amount','price_per_kg','price','product_id','type','weight', 'data-old-price-per-kg',
                                      'data-old-price','data-group-quantity', 'data-energy-drink','datetm', '_id', 'url'])
    w.writeheader()
    for row in db.prisma.find():
        w.writerow(row)

In [74]:
list(islice(db.lenta.find({'data-item-weight': {'$exists': 1 }}), 5))

[{'_id': ObjectId('5e6d415f391ce6c0a6d117d4'),
  'data-item-weight': '0',
  'name': 'Груша Санта Мария',
  'url': '/products/162416-grusha-santa-mariya',
  'amount': '0',
  'price': '21.06',
  'type': 'weight',
  'weight': '100',
  'datetm': datetime.datetime(2020, 3, 14, 23, 41, 3, 187000),
  'price_per_kg': '210.6',
  'product_id': '4860126'},
 {'_id': ObjectId('5e6d4161391ce6c0a6d117f5'),
  'data-item-weight': '0',
  'name': 'Фисташки жареные соленые калиброванные 24/26',
  'url': '/products/29875-fistashki-zharenye-solenye-kalibrovannye-24-26',
  'amount': '0',
  'price': '137.64',
  'type': 'weight',
  'weight': '100',
  'datetm': datetime.datetime(2020, 3, 14, 23, 41, 5, 761000),
  'price_per_kg': '1376.4',
  'product_id': '609744'},
 {'_id': ObjectId('5e6d4162391ce6c0a6d11804'),
  'data-item-weight': '0',
  'name': 'Лайм',
  'url': '/products/39897-laym',
  'amount': '0',
  'price': '73.7',
  'type': 'weight',
  'weight': '200',
  'datetm': datetime.datetime(2020, 3, 14, 23, 41,

In [ ]:
list(islice(db.shop_name.find(), 5))

In [15]:
list(db.prisma.find({'name': 'Чурчхела ТМ Семушка'}))

[{'_id': ObjectId('5e6d3600b34c577ef039ef67'),
  'data-amount': '0',
  'data-price-per-kg': '11990.0',
  'data-price': '119.9',
  'data-product-id': '4047135',
  'data-type': 'piece',
  'data-weight': '10',
  'name': 'Микро зелень Кервель ТМ Новагрин',
  'url': '/products/138039-mikro-zelen-kervel-tm-novagrin',
  'insert_datetm': datetime.datetime(2020, 3, 14, 22, 52, 32, 366000)},
 {'_id': ObjectId('5e6d3600b34c577ef039ef68'),
  'data-amount': '0',
  'data-price-per-kg': '9997.8',
  'data-price': '499.89',
  'data-product-id': '137980',
  'data-type': 'piece',
  'data-weight': '50',
  'name': 'Кешью обжаренный с солью ТМ Rioba (Риоба)',
  'url': '/products/31423-keshyu-obzharennyy-s-solyu-tm-rioba-rioba',
  'insert_datetm': datetime.datetime(2020, 3, 14, 22, 52, 32, 369000)},
 {'_id': ObjectId('5e6d3600b34c577ef039ef69'),
  'data-amount': '0',
  'data-price-per-kg': '3132.6',
  'data-price': '469.89',
  'data-product-id': '669869',
  'data-type': 'piece',
  'data-weight': '150',
  'na

In [66]:
it = db.carusel.aggregate([
    {'$group': {
        '_id': {'name': '$name'},
        'uniqueIds': {'$addToSet': "$_id"},
        'count': {'$sum': 1}
        }
    },
    {'$match': { 
        'count': {"$gt": 1}
        }
    },
    {'$sort': {
        'count': -1
        }
    }
])
list(it)

[{'_id': {'name': 'Колготки ТМ Omsa (Омса)'},
  'uniqueIds': [ObjectId('5e6d4de5391ce6c0a6d23e88'),
   ObjectId('5e6d4dc5391ce6c0a6d23989'),
   ObjectId('5e6d4dc9391ce6c0a6d23a59'),
   ObjectId('5e6d4dd1391ce6c0a6d23bd8'),
   ObjectId('5e6d4dde391ce6c0a6d23d60'),
   ObjectId('5e6d4de5391ce6c0a6d23e85'),
   ObjectId('5e6d4dc5391ce6c0a6d2398b'),
   ObjectId('5e6d4dc9391ce6c0a6d23a58'),
   ObjectId('5e6d4ddc391ce6c0a6d23ce6'),
   ObjectId('5e6d4de8391ce6c0a6d23f08'),
   ObjectId('5e6d4dc9391ce6c0a6d23a55'),
   ObjectId('5e6d4dcd391ce6c0a6d23b09'),
   ObjectId('5e6d4dc6391ce6c0a6d239d1'),
   ObjectId('5e6d4dc5391ce6c0a6d2398e'),
   ObjectId('5e6d4dd0391ce6c0a6d23b99'),
   ObjectId('5e6d4dcd391ce6c0a6d23b0a'),
   ObjectId('5e6d4dde391ce6c0a6d23d63'),
   ObjectId('5e6d4dcc391ce6c0a6d23aef'),
   ObjectId('5e6d4dd1391ce6c0a6d23bd6'),
   ObjectId('5e6d4dd4391ce6c0a6d23c75'),
   ObjectId('5e6d4dc6391ce6c0a6d239d2'),
   ObjectId('5e6d4dcc391ce6c0a6d23ae7'),
   ObjectId('5e6d4dc9391ce6c0a6d23a3a')

In [40]:
it = db.prisma.aggregate([
    {'$group': {
        '_id': {'name': '$name', 'data-weight': ''},
        'uniqueIds': {'$addToSet': "$_id"},
        'count': {'$sum': 1}
        }
    },
    {'$match': { 
        'count': {"$gt": 1}
        }
    },
    {'$sort': {
        'count': -1
        }
    }
])
list(it)

[{'_id': {'name': 'Колготки ТМ Omsa (Омса)', 'data-weight': '120'},
  'uniqueIds': [ObjectId('5e6d4136391ce6c0a6d11538'),
   ObjectId('5e6d4137391ce6c0a6d11568'),
   ObjectId('5e6d4136391ce6c0a6d11545'),
   ObjectId('5e6d4137391ce6c0a6d11580'),
   ObjectId('5e6d4137391ce6c0a6d11592'),
   ObjectId('5e6d4137391ce6c0a6d11562'),
   ObjectId('5e6d4135391ce6c0a6d11521'),
   ObjectId('5e6d4137391ce6c0a6d11581'),
   ObjectId('5e6d4136391ce6c0a6d11546'),
   ObjectId('5e6d4138391ce6c0a6d115b2'),
   ObjectId('5e6d4134391ce6c0a6d114ec'),
   ObjectId('5e6d4135391ce6c0a6d1151c'),
   ObjectId('5e6d4136391ce6c0a6d1153e'),
   ObjectId('5e6d4137391ce6c0a6d11589'),
   ObjectId('5e6d4137391ce6c0a6d1155e'),
   ObjectId('5e6d4137391ce6c0a6d11577')],
  'count': 16},
 {'_id': {'name': 'Носки женские ТМ MiNiMi (МиНиМи)', 'data-weight': '25'},
  'uniqueIds': [ObjectId('5e6d4137391ce6c0a6d11598'),
   ObjectId('5e6d4137391ce6c0a6d1156e'),
   ObjectId('5e6d4137391ce6c0a6d11573'),
   ObjectId('5e6d4135391ce6c0a6d11

In [34]:
import json
with open('/home/comp/Documents/src/food/data_20_03_15-16_35_58_.json', encoding='utf-8') as r:
    with open('/home/comp/Documents/src/food/data_20_03_15-16_35_58__.json', 'w', encoding='utf-8') as w:
        c = r.read()
        data = json.loads(f'[{c}]')
        w.write(json.dumps(data, ensure_ascii=False))

JSONDecodeError: Expecting ',' delimiter: line 2 column 1 (char 461)